In [22]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import math

In [23]:
from datetime import datetime

In [24]:
def get_df(parquet):
    df = pd.read_parquet(parquet)

    actual_first_row = df.columns.to_list()
    new_headers = ["date_time", "A", "B", "C", "D", "E", "F"]
    df.columns = new_headers
    df.loc[-1] = actual_first_row
    df.index = df.index + 1
    df = df.sort_index()
    df.columns = new_headers

    return df

In [25]:
def date_encoding(date):
    date_object = datetime.strptime(date, "%Y-%m-%d")
    return date_object.weekday()/7

In [26]:
def time_encoding(time):
    splitted = time.split(":")
    hour = int(splitted[0])
    minute = int(splitted[1])
    second = int(splitted[2])
    millisecond = int(splitted[3])
    l = [hour, minute, second, millisecond]
    arr = np.array(l, dtype=np.float64)

    arr[1] /= 60
    arr[2] /= 3600
    arr[3] /= 3600000

    total_hours = arr[0] + arr[1] + arr[2] + arr[3]

    return total_hours/24

In [27]:
def actiography_features_encoding(df, feature):
    df[feature] = df[feature].astype(float)
    mean = df[feature].mean()
    sd = math.sqrt(df[feature].var())
    df[feature] = (df[feature]-mean)/sd
    return df

In [30]:
def encode_df_to_array(df):
    df["date_encoded"] = df["date_only"].apply(date_encoding)
    df["time_encoded"] = df["time"].apply(time_encoding)
    actiography_features = ["Accelerometer_X", "Accelerometer_Y", "Accelerometer_Z", "Light_Lux", "Temperature_C"]

    for f in actiography_features:
        df = actiography_features_encoding(df=df, feature=f)

    return df

In [33]:
df = pd.read_csv("minute_data/combined_sorted_filtered.csv")
df = encode_df_to_array(df)
print(df)
df.to_csv("minute_data/encoded.csv")

        Unnamed: 0.1  Unnamed: 0                     date  Accelerometer_X  \
0                  0       39524  2024-08-20 00:00:34:000         0.585266   
1                  1       39525  2024-08-20 00:01:34:000         0.586307   
2                  2       39526  2024-08-20 00:02:34:000         0.587348   
3                  3       39527  2024-08-20 00:03:34:000         0.588041   
4                  4       39528  2024-08-20 00:04:34:000         0.588908   
...              ...         ...                      ...              ...   
180967        180967      220491  2025-02-28 02:57:05:000        -1.500941   
180968        180968      220492  2025-02-28 02:58:05:000        -1.500074   
180969        180969      220493  2025-02-28 02:59:05:000        -1.500594   
180970        180970      220494  2025-02-28 03:00:05:000        -1.499727   
180971        180971      220495  2025-02-28 03:01:05:000        -1.500074   

        Accelerometer_Y  Accelerometer_Z  Light_Lux  Temperatur

In [11]:
df = pd.read_parquet("parquet/DD_07_2024_007/Month 1/DD072024007_left wrist_101698_2024-08-20 13-09-13.parquet")
print(df['0'])

0            0
1            0
2            0
3            0
4            0
            ..
120938394    0
120938395    0
120938396    0
120938397    0
120938398    0
Name: 0, Length: 120938399, dtype: int64


In [3]:
df_m1 = pd.read_parquet("cleaned_parquet/007/Month 1/DD072024007_left wrist_101698_2024-08-20 13-09-13.parquet")
print(df_m1)

                         date_time         A         B         C         D  \
index                                                                        
0          2024-07-23 13:16:34:000 -0.691692 -0.069835  1.108325 -0.168220   
1          2024-07-23 13:16:34:020 -0.682757 -0.038617  1.108325 -0.168220   
2          2024-07-23 13:16:34:040 -0.673822 -0.030714  1.127044 -0.168220   
3          2024-07-23 13:16:34:060 -0.665111 -0.030714  1.127044 -0.168220   
4          2024-07-23 13:16:34:080 -0.665111 -0.062130  1.139366 -0.166802   
...                            ...       ...       ...       ...       ...   
120938395  2024-08-20 13:09:21:900  0.699042  2.226066 -0.596272 -0.196660   
120938396  2024-08-20 13:09:21:920  0.468743  1.724406 -0.453234 -0.195242   
120938397  2024-08-20 13:09:21:940  0.096599  1.849870 -0.440755 -0.193824   
120938398  2024-08-20 13:09:21:960  0.149763  1.269967 -0.789225 -0.193824   
120938399  2024-08-20 13:09:21:980  0.300318  0.956602 -0.894982

In [4]:
df_m2 = pd.read_parquet("cleaned_parquet/007/Month 2/DD072024007_left wrist_101697_2024-09-18 11-15-01.parquet")
print(df_m2)

                       date_time         A         B         C         D    E  \
index                                                                           
0        2024-08-20 13:13:48:000  1.539785 -0.702989 -1.288003 -0.102814  0.0   
1        2024-08-20 13:13:48:020  1.715609 -0.899471 -1.261651 -0.113405  0.0   
2        2024-08-20 13:13:48:040  1.659757 -0.825892 -1.221955 -0.121347  0.0   
3        2024-08-20 13:13:48:060  1.739603 -0.801230 -1.221955 -0.121347  0.0   
4        2024-08-20 13:13:48:080  1.731739 -0.809383 -1.307851 -0.129290  0.0   
...                          ...       ...       ...       ...       ...  ...   
8170495  2024-08-22 10:37:17:900 -0.425730  0.680335 -0.977277  2.837380  0.0   
8170496  2024-08-22 10:37:17:920 -0.489647  0.655673 -0.997125  2.837380  0.0   
8170497  2024-08-22 10:37:17:940 -0.425730  0.655673 -0.970773  2.837380  0.0   
8170498  2024-08-22 10:37:17:960 -0.425730  0.704793 -0.990620  2.837380  0.0   
8170499  2024-08-22 10:37:17

In [5]:
df_m3 = pd.read_parquet("./cleaned_parquet/007/Month 3/DD072024007_left wrist_101696_2024-10-22 15-22-36.parquet")
print(df_m3)

                         date_time         A         B         C         D  \
index                                                                        
0          2024-09-18 11:07:12:000 -1.142986 -0.077163  1.027523 -0.067997   
1          2024-09-18 11:07:12:020 -0.990784 -0.008477  1.002565 -0.067997   
2          2024-09-18 11:07:12:040 -0.904758 -0.260584  1.002565 -0.067997   
3          2024-09-18 11:07:12:060 -0.865053 -0.314179  1.065039 -0.071047   
4          2024-09-18 11:07:12:080 -0.931227 -0.337010  1.140071 -0.074287   
...                            ...       ...       ...       ...       ...   
128188795  2024-10-18 03:16:47:900 -1.579906  0.457623 -0.141280 -0.106497   
128188796  2024-10-18 03:16:47:920 -1.586524  0.472908 -0.147558 -0.106497   
128188797  2024-10-18 03:16:47:940 -1.586524  0.450077 -0.147558 -0.106497   
128188798  2024-10-18 03:16:47:960 -1.599759  0.442337 -0.153837 -0.106497   
128188799  2024-10-18 03:16:47:980 -1.573289  0.450077 -0.153837

In [6]:
df_m4 = pd.read_parquet("cleaned_parquet/007/Month 4/DD072024007_left wrist_101697_2024-11-20 13-12-43.parquet")
print(df_m4)

                        date_time         A         B         C         D  \
index                                                                       
0         2024-10-22 12:36:53:000 -0.224456 -0.924466 -1.191618  0.089199   
1         2024-10-22 12:36:53:020 -0.312303 -0.939263 -1.150097  0.076695   
2         2024-10-22 12:36:53:040 -0.341462 -0.953877 -1.191618  0.067317   
3         2024-10-22 12:36:53:060 -0.363424 -0.961184 -1.143118  0.067317   
4         2024-10-22 12:36:53:080 -0.334264 -0.939263 -1.115554  0.070443   
...                           ...       ...       ...       ...       ...   
35360695  2024-10-30 17:03:46:900  2.013428  1.078245  0.675431 -0.100656   
35360696  2024-10-30 17:03:46:920  2.042772  1.100350  0.647692 -0.100656   
35360697  2024-10-30 17:03:46:940  2.035390  1.063631  0.640888 -0.100656   
35360698  2024-10-30 17:03:46:960  2.028193  1.085735  0.675431 -0.100656   
35360699  2024-10-30 17:03:46:980  2.064734  1.056324  0.689213 -0.100656   

In [2]:
df_m5 = pd.read_parquet("cleaned_parquet/007/Month 5/DD072024007_left wrist_101699_2024-11-19 10-32-50.parquet")
print(df_m5)

                         date_time         A         B         C         D  \
index                                                                        
0          2024-10-16 17:35:05:000  0.986177 -0.409883 -1.520246  0.349904   
1          2024-10-16 17:35:05:020  1.543891 -0.476043 -1.685520  0.361813   
2          2024-10-16 17:35:05:040  0.979676 -0.417172 -2.009049  0.403664   
3          2024-10-16 17:35:05:060  0.261733 -0.218694 -2.064140  0.403664   
4          2024-10-16 17:35:05:080  0.563016 -0.380355 -2.022909  0.421698   
...                            ...       ...       ...       ...       ...   
117653095  2024-11-12 23:12:46:900 -1.135965 -0.042269  0.394259 -0.079160   
117653096  2024-11-12 23:12:46:920 -1.180816 -0.056847  0.407944 -0.079160   
117653097  2024-11-12 23:12:46:940 -1.187154 -0.005452  0.401102 -0.079160   
117653098  2024-11-12 23:12:46:960 -1.193654 -0.012741  0.380399 -0.079160   
117653099  2024-11-12 23:12:46:980 -1.212830 -0.020216  0.414787

In [2]:
df_m6 = pd.read_parquet("cleaned_parquet/007/Month 6/DD072024007_left wrist_101696_2025-01-30 13-12-54.parquet")
print(df_m6)

                         date_time         A         B         C         D  \
index                                                                        
0          2024-12-18 13:48:46:000 -0.147747 -0.066134 -1.554510  0.541125   
1          2024-12-18 13:48:46:020 -0.154546 -0.073998 -1.593310  0.541125   
2          2024-12-18 13:48:46:040 -0.195339 -0.073998 -1.645098  0.541125   
3          2024-12-18 13:48:46:060 -0.161345 -0.112731 -1.619285  0.541125   
4          2024-12-18 13:48:46:080 -0.174943 -0.112731 -1.599804  0.541125   
...                            ...       ...       ...       ...       ...   
124112395  2025-01-16 07:19:33:900 -1.664244  0.384306  1.102389 -0.063687   
124112396  2025-01-16 07:19:33:920 -1.643848  0.384306  1.179989 -0.063687   
124112397  2025-01-16 07:19:33:940 -1.725434  0.454104  1.147683 -0.063687   
124112398  2025-01-16 07:19:33:960 -1.725434  0.523901  1.167002 -0.063687   
124112399  2025-01-16 07:19:33:980 -1.637049  0.423039  1.121870

In [2]:
df_m7 = pd.read_parquet("cleaned_parquet/007/Month 7/DD072024007__100889_2025-03-26 12-05-58.parquet")
print(df_m7)

                         date_time         A         B         C         D  \
index                                                                        
0          2025-01-30 12:10:05:000  2.441999 -0.365826 -0.354713  0.628275   
1          2025-01-30 12:10:05:020  2.505794 -0.480054 -0.445056  0.633710   
2          2025-01-30 12:10:05:040  2.473996 -0.569790 -0.451497  0.633710   
3          2025-01-30 12:10:05:060  2.665382 -0.586050 -0.451497  0.639145   
4          2025-01-30 12:10:05:080  2.649483 -0.259625 -0.516074  0.639145   
...                            ...       ...       ...       ...       ...   
123636295  2025-02-28 03:02:10:900 -1.003160 -0.104645  0.897695 -0.064116   
123636296  2025-02-28 03:02:10:920 -1.074906 -0.063893  0.904136 -0.064116   
123636297  2025-02-28 03:02:10:940 -1.034959 -0.104645  0.897695 -0.064116   
123636298  2025-02-28 03:02:10:960 -1.011110 -0.088179  0.891253 -0.064116   
123636299  2025-02-28 03:02:10:980 -1.003160 -0.120904  0.891253

In [8]:
print(df_m7.loc["2025-01-30"])

                          date_time         A         B         C         D  \
date                                                                          
2025-01-30  2025-01-30 12:10:05:000  2.441999 -0.365826 -0.354713  0.628275   
2025-01-30  2025-01-30 12:10:05:020  2.505794 -0.480054 -0.445056  0.633710   
2025-01-30  2025-01-30 12:10:05:040  2.473996 -0.569790 -0.451497  0.633710   
2025-01-30  2025-01-30 12:10:05:060  2.665382 -0.586050 -0.451497  0.639145   
2025-01-30  2025-01-30 12:10:05:080  2.649483 -0.259625 -0.516074  0.639145   
...                             ...       ...       ...       ...       ...   
2025-01-30  2025-01-30 23:59:59:900  0.065465  0.539973 -1.258468 -0.064116   
2025-01-30  2025-01-30 23:59:59:920  0.057515  0.523713 -1.284398 -0.064116   
2025-01-30  2025-01-30 23:59:59:940  0.065465  0.548206 -1.284398 -0.064116   
2025-01-30  2025-01-30 23:59:59:960  0.097462  0.515481 -1.316605 -0.064116   
2025-01-30  2025-01-30 23:59:59:980  0.153308  0.515

In [6]:
print(df_m7.columns)

Index(['date_time', 'A', 'B', 'C', 'D', 'E', 'F', 'date_encoded',
       'time_encoded'],
      dtype='object')


In [2]:
df_3 = pd.read_parquet("./cleaned_parquet/007/Month 3/DD072024007_left wrist_101696_2024-10-22 15-22-36.parquet")
df_3["date"] = pd.to_datetime(df_3["date"])
neutral_start_date = pd.to_datetime('2024-09-26')
neutral_end_date = pd.to_datetime('2024-10-15')


In [3]:
df_3.set_index([df_3.index, 'date'])

,,date_time,A,B,C,D,E,F,date_encoded,time_encoded
index,date,,,,,,,,,
0,2024-09-18,2024-09-18 11:07:12:000,-1.142986,-0.077163,1.027523,-0.067997,0.0,1.534327,0.285714,0.463333
1,2024-09-18,2024-09-18 11:07:12:020,-0.990784,-0.008477,1.002565,-0.067997,0.0,1.534327,0.285714,0.463334
2,2024-09-18,2024-09-18 11:07:12:040,-0.904758,-0.260584,1.002565,-0.067997,0.0,1.534327,0.285714,0.463334
3,2024-09-18,2024-09-18 11:07:12:060,-0.865053,-0.314179,1.065039,-0.071047,0.0,1.534327,0.285714,0.463334
4,2024-09-18,2024-09-18 11:07:12:080,-0.931227,-0.337010,1.140071,-0.074287,0.0,1.534327,0.285714,0.463334
...,...,...,...,...,...,...,...,...,...,...
128188795,2024-10-18,2024-10-18 03:16:47:900,-1.579906,0.457623,-0.141280,-0.106497,0.0,0.790639,0.571429,0.136666
128188796,2024-10-18,2024-10-18 03:16:47:920,-1.586524,0.472908,-0.147558,-0.106497,0.0,0.790639,0.571429,0.136666
128188797,2024-10-18,2024-10-18 03:16:47:940,-1.586524,0.450077,-0.147558,-0.106497,0.0,0.790639,0.571429,0.136666


In [4]:
df_3.loc[neutral_start_date <= df_3["date"]] 

,date_time,A,B,C,D,E,F,date_encoded,time_encoded,date
index,,,,,,,,,,
32558400,2024-09-26 00:00:00:000,0.326426,-0.971246,-1.435188,-0.106497,0.0,1.056242,0.428571,0.000000e+00,2024-09-26
32558401,2024-09-26 00:00:00:020,0.333043,-0.994077,-1.435188,-0.106497,0.0,1.056242,0.428571,2.314815e-07,2024-09-26
32558402,2024-09-26 00:00:00:040,0.346278,-0.963506,-1.428909,-0.106497,0.0,1.056242,0.428571,4.629630e-07,2024-09-26
32558403,2024-09-26 00:00:00:060,0.346278,-1.009362,-1.435188,-0.106497,0.0,1.056242,0.428571,6.944444e-07,2024-09-26
32558404,2024-09-26 00:00:00:080,0.313191,-0.994077,-1.454025,-0.106497,0.0,1.056242,0.428571,9.259259e-07,2024-09-26
...,...,...,...,...,...,...,...,...,...,...
128188795,2024-10-18 03:16:47:900,-1.579906,0.457623,-0.141280,-0.106497,0.0,0.790639,0.571429,1.366655e-01,2024-10-18
128188796,2024-10-18 03:16:47:920,-1.586524,0.472908,-0.147558,-0.106497,0.0,0.790639,0.571429,1.366657e-01,2024-10-18
128188797,2024-10-18 03:16:47:940,-1.586524,0.450077,-0.147558,-0.106497,0.0,0.790639,0.571429,1.366660e-01,2024-10-18


In [5]:
df_3.loc[neutral_end_date >= df_3["date"]] 

,date_time,A,B,C,D,E,F,date_encoded,time_encoded,date
index,,,,,,,,,,
0,2024-09-18 11:07:12:000,-1.142986,-0.077163,1.027523,-0.067997,0.0,1.534327,0.285714,0.463333,2024-09-18
1,2024-09-18 11:07:12:020,-0.990784,-0.008477,1.002565,-0.067997,0.0,1.534327,0.285714,0.463334,2024-09-18
2,2024-09-18 11:07:12:040,-0.904758,-0.260584,1.002565,-0.067997,0.0,1.534327,0.285714,0.463334,2024-09-18
3,2024-09-18 11:07:12:060,-0.865053,-0.314179,1.065039,-0.071047,0.0,1.534327,0.285714,0.463334,2024-09-18
4,2024-09-18 11:07:12:080,-0.931227,-0.337010,1.140071,-0.074287,0.0,1.534327,0.285714,0.463334,2024-09-18
...,...,...,...,...,...,...,...,...,...,...
118958395,2024-10-15 23:59:59:900,0.180842,0.755586,1.358888,-0.106497,0.0,-0.059291,0.142857,0.999999,2024-10-15
118958396,2024-10-15 23:59:59:920,0.167607,0.755586,1.352609,-0.106497,0.0,-0.059291,0.142857,0.999999,2024-10-15
118958397,2024-10-15 23:59:59:940,0.160990,0.748040,1.358888,-0.106497,0.0,-0.059291,0.142857,0.999999,2024-10-15


In [2]:
df_6 = pd.read_parquet("cleaned_parquet/007/Month 6/DD072024007_left wrist_101696_2025-01-30 13-12-54.parquet")
df_6["date"] = pd.to_datetime(df_6["date"])
good_start_date = pd.to_datetime('2025-01-01')
good_end_date = pd.to_datetime('2025-01-13')


In [3]:
df_6.set_index([df_6.index, 'date'])

,,date_time,A,B,C,D,E,F,date_encoded,time_encoded
index,date,,,,,,,,,
0,2024-12-18,2024-12-18 13:48:46:000,-0.147747,-0.066134,-1.554510,0.541125,0.0,0.705443,0.285714,0.575532
1,2024-12-18,2024-12-18 13:48:46:020,-0.154546,-0.073998,-1.593310,0.541125,0.0,0.705443,0.285714,0.575533
2,2024-12-18,2024-12-18 13:48:46:040,-0.195339,-0.073998,-1.645098,0.541125,0.0,0.705443,0.285714,0.575533
3,2024-12-18,2024-12-18 13:48:46:060,-0.161345,-0.112731,-1.619285,0.541125,0.0,0.705443,0.285714,0.575533
4,2024-12-18,2024-12-18 13:48:46:080,-0.174943,-0.112731,-1.599804,0.541125,0.0,0.705443,0.285714,0.575533
...,...,...,...,...,...,...,...,...,...,...
124112395,2025-01-16,2025-01-16 07:19:33:900,-1.664244,0.384306,1.102389,-0.063687,0.0,-0.381763,0.428571,0.305253
124112396,2025-01-16,2025-01-16 07:19:33:920,-1.643848,0.384306,1.179989,-0.063687,0.0,-0.381763,0.428571,0.305254
124112397,2025-01-16,2025-01-16 07:19:33:940,-1.725434,0.454104,1.147683,-0.063687,0.0,-0.381763,0.428571,0.305254


In [4]:
df_6.loc[good_start_date <= df_6["date"]] 

,date_time,A,B,C,D,E,F,date_encoded,time_encoded,date
index,,,,,,,,,,
57993700,2025-01-01 00:00:00:000,-1.276709,-0.982349,0.947352,-0.063687,0.0,0.905717,0.285714,0.000000e+00,2025-01-01
57993701,2025-01-01 00:00:00:020,-1.242714,-0.974682,0.953683,-0.063687,0.0,0.905717,0.285714,2.314815e-07,2025-01-01
57993702,2025-01-01 00:00:00:040,-1.263111,-0.974682,0.973165,-0.063687,0.0,0.905717,0.285714,4.629630e-07,2025-01-01
57993703,2025-01-01 00:00:00:060,-1.256312,-0.974682,0.960177,-0.063687,0.0,0.905717,0.285714,6.944444e-07,2025-01-01
57993704,2025-01-01 00:00:00:080,-1.249513,-0.982349,0.947352,-0.063687,0.0,0.905717,0.285714,9.259259e-07,2025-01-01
...,...,...,...,...,...,...,...,...,...,...
124112395,2025-01-16 07:19:33:900,-1.664244,0.384306,1.102389,-0.063687,0.0,-0.381763,0.428571,3.052535e-01,2025-01-16
124112396,2025-01-16 07:19:33:920,-1.643848,0.384306,1.179989,-0.063687,0.0,-0.381763,0.428571,3.052537e-01,2025-01-16
124112397,2025-01-16 07:19:33:940,-1.725434,0.454104,1.147683,-0.063687,0.0,-0.381763,0.428571,3.052539e-01,2025-01-16


In [5]:
df_6.loc[good_end_date >= df_6["date"]] 

,date_time,A,B,C,D,E,F,date_encoded,time_encoded,date
index,,,,,,,,,,
0,2024-12-18 13:48:46:000,-0.147747,-0.066134,-1.554510,0.541125,0.0,0.705443,0.285714,0.575532,2024-12-18
1,2024-12-18 13:48:46:020,-0.154546,-0.073998,-1.593310,0.541125,0.0,0.705443,0.285714,0.575533,2024-12-18
2,2024-12-18 13:48:46:040,-0.195339,-0.073998,-1.645098,0.541125,0.0,0.705443,0.285714,0.575533,2024-12-18
3,2024-12-18 13:48:46:060,-0.161345,-0.112731,-1.619285,0.541125,0.0,0.705443,0.285714,0.575533,2024-12-18
4,2024-12-18 13:48:46:080,-0.174943,-0.112731,-1.599804,0.541125,0.0,0.705443,0.285714,0.575533,2024-12-18
...,...,...,...,...,...,...,...,...,...,...
114153695,2025-01-13 23:59:59:900,-2.133541,0.151320,0.287916,-0.063687,0.0,0.047397,0.000000,0.999999,2025-01-13
114153696,2025-01-13 23:59:59:920,-2.126742,0.151320,0.281423,-0.063687,0.0,0.047397,0.000000,0.999999,2025-01-13
114153697,2025-01-13 23:59:59:940,-2.126742,0.158988,0.274929,-0.063687,0.0,0.047397,0.000000,0.999999,2025-01-13
